In [1]:
import numpy as np
import scipy.sparse as sp

In [2]:
path_dimenet_data = '/home/kotobiam/dimenet/data/qm9_eV.npz'

In [3]:
data = np.load(path_dimenet_data, allow_pickle=True)

In [4]:
for k in data:
    print(k)

R
N
Z
id
A
B
C
mu
alpha
homo
lumo
gap
r2
zpve
U0
U
H
G
Cv
meta


In [5]:
data['N']

array([ 5,  4,  3, ..., 17, 18, 16])

In [18]:
np.repeat(np.arange(len([1, 2, 3, 4, 5])), [1, 3, 4, 5, 8])

array([0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4])

In [23]:
N

array([5, 4, 3])

In [8]:
np.zeros(np.sum(N[[0, 1, 2]]), dtype=np.int32)

NameError: name 'N' is not defined

In [17]:
N = data['N'][[0, 1, 2]]
N_cumsum = np.concatenate([[0], np.cumsum(data['N'])])
Z = np.zeros(np.sum(N), dtype=np.int32)
R = np.zeros([np.sum(N), 3], dtype=np.float32)
adj_matrices=[]

In [18]:
def bmat_fast(mats):
    new_data = np.concatenate([mat.data for mat in mats])

    ind_offset = np.zeros(1 + len(mats))

    ind_offset[1:] = np.cumsum([mat.shape[0] for mat in mats])

    new_indices = np.concatenate(
        [mats[i].indices + ind_offset[i] for i in range(len(mats))]
    )

    indptr_offset = np.zeros(1 + len(mats))
    indptr_offset[1:] = np.cumsum([mat.nnz for mat in mats])
    new_indptr = np.concatenate(
        [adj_matrices[i].indptr[i >= 1:] + indptr_offset[i] for i in range(len(mats))]
    )
  
    return sp.csr_matrix((new_data, new_indices, new_indptr))

In [19]:
nend=0
for k, i in enumerate([0, 1, 2]):
    n = N[k]
    nstart=nend
    nend=nstart+n
    
    Z[nstart:nend] = data['Z'][N_cumsum[i]:N_cumsum[i+1]]
    positions = data['R'][N_cumsum[i]:N_cumsum[i+1]]
    R[nstart:nend] = positions
    
    Dij = np.linalg.norm(positions[:, None, :] - positions[None, :, :], axis=-1)
    
    adj_matrices.append(sp.csr_matrix(Dij <= 3))

    adj_matrices[-1] -= sp.eye(n, dtype=np.bool)
    
adj_matrix = bmat_fast(adj_matrices)

atomids_to_edgeid = sp.csr_matrix(
    (np.arange(adj_matrix.nnz), adj_matrix.indices, adj_matrix.indptr), 
    shape=adj_matrix.shape)



In [25]:
target, source = adj_matrix.nonzero()

In [39]:
source

array([ 1,  2,  3,  4,  0,  2,  3,  4,  0,  1,  3,  4,  0,  1,  2,  4,  0,
        1,  2,  3,  6,  7,  8,  5,  7,  8,  5,  6,  8,  5,  6,  7, 10, 11,
        9, 11,  9, 10], dtype=int32)

In [34]:
adj_matrix[source].sum(1).A1

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2])

In [38]:
adj_matrix.data.shape

(38,)

In [42]:
adj_matrix.data

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

# Understanding sparse matrix

In [33]:
Dij <= 1

array([[ True,  True,  True],
       [ True,  True, False],
       [ True, False,  True]])

In [31]:
sp.csr_matrix(Dij <= 1).data

array([ True,  True,  True,  True,  True,  True,  True])

In [32]:
sp.csr_matrix(Dij <= 1).indices

array([0, 1, 2, 0, 1, 0, 2], dtype=int32)

In [12]:
sp.csr_matrix(Dij <= 1).indptr[i >= 1:]

array([3, 5, 7], dtype=int32)

In [11]:
sp.csr_matrix(Dij <= 1).nnz

7

In [31]:
import dgl 

edgeid_to_target, edgeid_to_source = adj_matrix.nonzero()
test_g = dgl.graph((edgeid_to_target, edgeid_to_source))

In [32]:
test_g

Graph(num_nodes=12, num_edges=38,
      ndata_schemes={}
      edata_schemes={})

In [26]:
sp.csr_matrix(np.arange(adj_matrix.nnz)).toarray()

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37]])

In [15]:
adj_matrices

new_data = np.concatenate([mat.data for mat in adj_matrices])

In [16]:
adj_matrices[0].toarray()

array([[False,  True,  True,  True,  True],
       [ True, False,  True,  True,  True],
       [ True,  True, False,  True,  True],
       [ True,  True,  True, False,  True],
       [ True,  True,  True,  True, False]])

In [17]:
test_mat = [[0. , 1.0919529 , 1.0919516 , 1.0919464], 
            [0. , 1.0919529 , 0 , 1.0919464],
           [0. , 1.0919529 , 1.0919516 , 1.0919464]]
test_mat = np.asarray(test_mat)
test_sp_mat = sp.csr_matrix(test_mat)

In [18]:
test_sp_mat.indices

array([1, 2, 3, 1, 3, 1, 2, 3], dtype=int32)

In [19]:
test_sp_mat.toarray()

array([[0.       , 1.0919529, 1.0919516, 1.0919464],
       [0.       , 1.0919529, 0.       , 1.0919464],
       [0.       , 1.0919529, 1.0919516, 1.0919464]])

In [211]:
test_sp_mat.indptr

array([0, 3, 5, 8], dtype=int32)

In [205]:
test_mat = np.asarray(test_mat)

In [207]:
test_mat.shape

(3, 4)